#### model = Concat signatures with pose -> Dense -> softmax with 100 classes. 
#### run model for each frame in a seq, and then take majority vote

In [63]:
from data import *

In [64]:
pose_paths, keypoints, scores = read_pose('data/pose.pkl')
signatures_paths, signatures = read_signatures('data/signatures.pkl')

# X, y = load_data()

In [55]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

IMAGE_DIM = 64
NUM_CLASSES = 100
learning_rate = 1e-4
epochs = 500
INPUT_DIM = 2048 # signatures[0].shape[0]


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(IMAGE_DIM * IMAGE_DIM, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, NUM_CLASSES)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)
    
net = Net()
print(net)

optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = nn.NLLLoss()

Net(
  (fc1): Linear(in_features=4096, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
)


In [58]:
from tqdm import tqdm
for t in tqdm(range(epochs)):
    y_pred = net(x)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    model.zero_grad()
    loss.backward()
    optimizer.step()

  0%|                                               | 0/500 [00:00<?, ?it/s]


NameError: name 'x' is not defined